# Mimic of the DES night summary, but for Rubin Observatory

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2027-11-10"

In [ ]:
import datetime
import sys
import pandas as pd
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.transform
import bokeh.layouts
import sqlite3
import numpy as np
import healpy
import astropy
import colorcet
import healpy as hp
from astropy.visualization import ZScaleInterval
from astropy.time import Time
from lsst.resources import ResourcePath

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_scheduler.site_models
import schedview.compute.astro
import uranography
import schedview.plot.visitmap

from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from uranography.api import Planisphere, make_zscale_linear_cmap

In [ ]:
bokeh.io.output_notebook()

In [ ]:
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
baseline_opsim_rp = ResourcePath('/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.4/baseline/baseline_v3.4_10yrs.db')
day_obs_mjd = int(Time(day_obs).mjd)

In [ ]:
band_cmap = bokeh.transform.factor_cmap(
    'filter',
    ('#56b4e9', '#008060', '#ff4000', '#850000', '#6600cc', '#000000'),
    ['u', 'g', 'r', 'i', 'z', 'y'])

In [ ]:
fiducial_depth = {
                "u": 23.71,
                "g": 24.67,
                "r": 24.24,
                "i": 23.82,
                "z": 23.21,
                "y": 22.40,
}

In [ ]:
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

In [ ]:
def visit_query(visit_resource_path, query):
    with baseline_opsim_rp.as_local() as local_baseline_opsim_rp:
        with sqlite3.connect(local_baseline_opsim_rp.ospath) as baseline_db_connection:
            visits = pd.read_sql_query(query, baseline_db_connection)

    visits.set_index('observationId', inplace=True)
    
    # Add start time
    start_time = pd.to_datetime(visits['observationStartMJD'] + 2400000.5, origin='julian', unit='D')
    visits.insert(0, 'start_time', start_time)

    # Add day_obs
    day_obs_mjd = np.floor(visits['observationStartMJD'] - 0.5).astype('int')
    day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
    day_obs_date = [datetime.date(t.year, t.month, t.day) for t in day_obs_datetime]
    day_obs_iso8601 = tuple(str(d) for d in day_obs_date)
    visits.insert(1, 'day_obs_mjd', day_obs_mjd)
    visits.insert(2, 'day_obs_date', day_obs_date)
    visits.insert(3, 'day_obs_iso8601', day_obs_iso8601)

    # Add coordinates tuple
    coord_column = max(tuple(visits.columns).index('fieldRA'), tuple(visits.columns).index('fieldDec')) + 1
    visits.insert(coord_column, 'coords', list(zip(visits['fieldRA'], visits['fieldDec'])))
    
    # Add derived depth columns
    depth_col_index = tuple(visits.columns).index('fiveSigmaDepth')
    tau = 10**(0.8*(visits.fiveSigmaDepth - visits['filter'].map(fiducial_depth)))
    t_eff = visits['visitExposureTime'] * tau
    visits.insert(depth_col_index+1, 'tau', tau)
    visits.insert(depth_col_index+2, 'teff', t_eff)

    # Add time between successive exposures
    # Assume the previous visits ended at its observationStartMJD + visitTime,
    # That this visit ends at its own observationStartMJD + visitTime,
    # and visitExposureTime does not count as overhead.
    overhead = visits['observationStartMJD'].diff()*24*60*60 - visits['visitTime'].shift(1) + visits['visitTime'] - visits['visitExposureTime']
    slew_time_col_index = tuple(visits.columns).index('slewTime')
    visits.insert(slew_time_col_index + 1, 'overhead', overhead)

    #
    filter_col_index = tuple(visits.columns).index('filter')
    previous_filter = visits['filter'].shift(1)
    visits.insert(filter_col_index + 1, 'previous_filter', previous_filter)
    
    return visits

In [ ]:
visits = visit_query(
    baseline_opsim_rp,
    f"SELECT * FROM observations WHERE FLOOR(observationStartMJD-0.5)={day_obs_mjd}"
)

Using fiducial depths for t_eff calculation from https://github.com/lsst-sims/smtn-002/blob/main/notebooks/teff_fiducial.ipynb commit e367d65.
These probably should be updated.

In [ ]:
visits.head()['start_time']

## Conditions and statistics

### Sun and Moon

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

In [ ]:
print(f"Moon phase: {visits['moonPhase'].median()}")

### Numbers of exposures, and gaps between them

In [ ]:
relative_start_time = (visits['observationStartMJD'].min() - night_events.loc['sun_n12_setting','MJD'])*60*24
print(f"Open shutter of first exposure: {relative_start_time} minutes before 12 degree evening twilight")

relative_end_time = ((visits['observationStartMJD'] + visits['visitTime']/(24*60*60)).max() - night_events.loc['sun_n12_rising','MJD'])*60*24
print(f"Close shutter time of last exposure: {-1*relative_end_time} minutes after 12 degree morning twilight")

total_time = ((visits['observationStartMJD'] + visits['visitTime']/(24*60*60)).max() - visits['observationStartMJD'].min())*24
print(f"Total wall clock time: {total_time} hours")

num_exposures = len(visits)
print(f"Number of exposures: {num_exposures}")

total_exptime = visits.visitExposureTime.sum()/(60*60)
print(f"Total open shutter time: {total_exptime} hours")

mean_gap_time = 60*60*(total_time - total_exptime)/(num_exposures - 1)
print(f"Mean gap time: {mean_gap_time} seconds")

median_gap_time = visits.overhead.median()
print(f"Median gap time: {median_gap_time} seconds")

## Histogram of gaps between exposures

In [ ]:
bins = np.arange(0, 30)
hist, edges = np.histogram(visits.overhead, density=False, bins=bins)
p1 = bokeh.plotting.figure(title='Overhead', y_axis_label='Overhead (seconds)')
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="skyblue", line_color="white")

p2 = bokeh.plotting.figure(title='Overhead vs. slew distance', y_axis_label='overhead (sec.)', x_axis_label='slew distance (deg.)')
for band in band_cmap.transform.factors:
    these_visits = visits.query(f'filter == "{band}"')
    if len(these_visits) > 0:
        p2.circle(x='slewDistance', y='overhead', color=band_cmap, fill_alpha=0.3, source=these_visits, legend_label=band)

legend = p2.legend[0]
legend.orientation = 'horizontal'
p2.add_layout(legend, 'below')

gap_plots = bokeh.layouts.row([p1, p2])

bokeh.io.show(gap_plots)

## Long gaps between exposures

In [ ]:
num_gaps = 10
long_gap_visits = visits.sort_values('overhead', ascending=False).query('overhead>30').loc[:, ['start_time', 'overhead', 'slewDistance', 'filter', 'previous_filter']].sort_values('observationId')
long_gap_visits

## PSF Width

In [ ]:
p = bokeh.plotting.figure(title='PSF Width', y_axis_label='seeingFwhmEff')
for band in band_cmap.transform.factors:
    these_visits = visits.query(f'filter == "{band}"')
    if len(these_visits) > 0:
        p.circle(x='start_time', y='seeingFwhmEff', color=band_cmap, fill_alpha=0.3, source=these_visits, legend_label=band)

p.xaxis[0].formatter = bokeh.models.DatetimeTickFormatter(hours="%H:%M")

legend = p.legend[0]
legend.orientation = 'horizontal'
p.add_layout(legend, 'below')
bokeh.io.show(p)

## Instrumental seeing

In [ ]:
# Get a seeing model that applies atmospheric and wavelength corrections, but not instrumental contributions.
seeing_model = rubin_scheduler.site_models.SeeingModel(
    telescope_seeing=0.0,
    optical_design_seeing=0.0,
    camera_seeing=0.0
)
seeing_indx_dict = {}
for i, filtername in enumerate(seeing_model.filter_list):
    seeing_indx_dict[filtername] = i
    
noninstrumental_seeing = np.array(tuple(seeing_model(v.seeingFwhm500, v.airmass)['fwhmEff'][seeing_indx_dict[v['filter']]].item() for i, v in visits.iterrows()))
visits['nonatmo_fwhm'] = np.sqrt(visits['seeingFwhmEff']**2 - noninstrumental_seeing**2)

In [ ]:
p = bokeh.plotting.figure(title='Instrumental seeing', y_axis_label='Instrumetal contribution to the FWHM')
for band in band_cmap.transform.factors:
    these_visits = visits.query(f'filter == "{band}"')
    if len(these_visits) > 0:
        p.circle(x='start_time', y='nonatmo_fwhm', color=band_cmap, fill_alpha=0.3, source=these_visits, legend_label=band)

p.xaxis[0].formatter = bokeh.models.DatetimeTickFormatter(hours="%H:%M")

legend = p.legend[0]
legend.orientation = 'horizontal'
p.add_layout(legend, 'below')
bokeh.io.show(p)

This perplexes me; I expected the instrumental contribution in simulations to be constant.

## PSF ellipticity

No ellipticity is simulated by opsim.

## Effective exposure time

In [ ]:
p = bokeh.plotting.figure(title='Effective exposure time', y_axis_label=r'Effecive exposure time (sec.)')
for band in band_cmap.transform.factors:
    these_visits = visits.query(f'filter == "{band}"')
    if len(these_visits) > 0:
        p.circle(x='start_time', y='teff', color=band_cmap, fill_alpha=0.3, source=these_visits, legend_label=band)

p.xaxis[0].formatter = bokeh.models.DatetimeTickFormatter(hours="%H:%M")

legend = p.legend[0]
legend.orientation = 'horizontal'
p.add_layout(legend, 'below')
bokeh.io.show(p)

## Sky brightness

When run with current opsim simulations, all simulations are either completely spoiled (infinite extinction) or clear (no extinction), and what is recorded is a fraction of the sky covered by clouds.

So, where the DES nightsum plots the extinction, what is plotted here is the recorded fraction cloud cover.

In [ ]:
p = bokeh.plotting.figure(title='Sky brightness', y_axis_label=r'cloud cover')
for band in band_cmap.transform.factors:
    these_visits = visits.query(f'filter == "{band}"')
    if len(these_visits) > 0:
        p.circle(x='start_time', y='cloud', color=band_cmap, fill_alpha=0.3, source=these_visits, legend_label=band)

p.xaxis[0].formatter = bokeh.models.DatetimeTickFormatter(hours="%H:%M")

legend = p.legend[0]
legend.orientation = 'horizontal'
p.add_layout(legend, 'below')
bokeh.io.show(p)

## Visit map

In [ ]:
vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
    visits=visits,
    night_date=day_obs_date,
    timezone=timezone,
    observatory=observatory,
)

In [ ]:
bokeh.io.show(vmap)

## Survey Progress

In [ ]:
def create_visit_map_for_band(visits, band, conditions, map_hpix, scale_limits=None, palette=colorcet.blues):
    camera_perimeter = LsstCameraFootprintPerimeter()

    plot = bokeh.plotting.figure(
        frame_width=256,
        frame_height=256,
        match_aspect=True,
        title=f"Visits in {band}",
    )
    psphere = Planisphere(mjd=conditions.mjd, plot=plot)

    if scale_limits is None:
        scale_limits = ZScaleInterval().get_limits(map_hpix[~ map_hpix.mask])

    cmap = bokeh.transform.linear_cmap('value', palette, scale_limits[0], scale_limits[1])
    psphere.add_healpix(map_hpix, nside=hp.npix2nside(len(map_hpix)), cmap=cmap)

    band_visits = visits.query(f"filter == '{band}'")

    if len(band_visits)>0:
        ras, decls = camera_perimeter(band_visits.fieldRA, band_visits.fieldDec, band_visits.rotSkyPos)
        
        perimeter_df = pd.DataFrame(
            {
                "ra": ras,
                "decl": decls,
            }
        )
        visit_ds = psphere.add_patches(
            perimeter_df,
            patches_kwargs={
                'fill_color': None,
                'line_color': 'black',
                'line_width': 1
            }
        )

    psphere.decorate()

    psphere.add_marker(
        ra=np.degrees(conditions.sun_ra),
        decl=np.degrees(conditions.sun_dec),
        name="Sun",
        glyph_size=8,
        circle_kwargs={"color": "yellow", "fill_alpha": 1},
    )

    psphere.add_marker(
        ra=np.degrees(conditions.moon_ra),
        decl=np.degrees(conditions.moon_dec),
        name="Moon",
        glyph_size=8,
        circle_kwargs={"color": "orange", "fill_alpha": 0.8},
    )

    return plot

### Map depth accumulated so far

In [ ]:
observatory.mjd = night_events.loc['night_middle', 'MJD']
conditions = observatory.return_conditions()

In [ ]:
bands = 'ugrizy'
bundle = {}
for band in bands:
    constraint = f"filter == '{band}' AND observationStartMjd < {night_events.loc['sunset', 'MJD']}"
    slicer = maf.HealpixSlicer(nside=32)
    metric = maf.Coaddm5Metric()
    bundle[band] = maf.MetricBundle(metric, slicer, constraint, run_name='last_night')

with baseline_opsim_rp.as_local() as opsim_db_rp:
    bundle_group = maf.MetricBundleGroup(
        bundle, opsim_db_rp.ospath
    )
    bundle_group.run_all()
    depth_hpix = {b: bundle[b].metric_values for b in bands}


In [ ]:
visit_map = {}
for band in 'ugrizy':
    tau_hpix = 10**(0.8*(depth_hpix[band] - fiducial_depth[band]))
    tau_hpix.fill_value = np.nan
    visit_map[band] = create_visit_map_for_band(visits, band, conditions, tau_hpix)

visit_map_grid = bokeh.layouts.gridplot([
    [visit_map['u'], visit_map['g'], visit_map['r']],
    [visit_map['i'], visit_map['z'], visit_map['y']]
])
bokeh.io.show(visit_map_grid)

### Map time since the most recent visits

In [ ]:
bands = 'ugrizy'
bundle = {}
for band in bands:
    constraint = f"filter == '{band}' AND observationStartMjd < {night_events.loc['sunset', 'MJD']}"
    slicer = maf.HealpixSlicer(nside=32)
    metric = maf.MaxMetric('observationStartMJD')
    bundle[band] = maf.MetricBundle(metric, slicer, constraint, run_name='last_night')

with baseline_opsim_rp.as_local() as opsim_db_rp:
    bundle_group = maf.MetricBundleGroup(
        bundle, opsim_db_rp.ospath
    )
    bundle_group.run_all()
    time_since_latest_hpix = {b: night_events.loc['sunset', 'MJD'] - bundle[b].metric_values for b in bands}

In [ ]:
visit_map = {band: create_visit_map_for_band(visits, band, conditions, time_since_latest_hpix[band], [0, 14], 'Viridis256') for band in 'ugrizy'}
visit_map_grid = bokeh.layouts.gridplot([
    [visit_map['u'], visit_map['g'], visit_map['r']],
    [visit_map['i'], visit_map['z'], visit_map['y']]
])
bokeh.io.show(visit_map_grid)

## DDF Cadence

In [ ]:
time_window_duration = 120
ddf_plot_mjds = np.arange(day_obs_mjd - time_window_duration, day_obs_mjd)

In [ ]:
ddf_plot_datetimes = Time(ddf_plot_mjds, format='mjd').datetime
ddf_plot_dates = [datetime.date(t.year, t.month, t.day) for t in ddf_plot_datetimes]
ddf_plot_iso8601 = [str(d) for d in ddf_plot_dates]

In [ ]:
ddf_field_names = tuple(rubin_scheduler.utils.ddf_locations().keys())

In [ ]:
ddf_visits = visit_query(
    baseline_opsim_rp,
    f"""SELECT * FROM observations
    WHERE target IN {tuple(field_name for field_name in ddf_field_names)}
      AND FLOOR(observationStartMJD-0.5)<={day_obs_mjd}
      AND FLOOR(observationStartMJD-0.5)>{day_obs_mjd-time_window_duration}""")

In [ ]:
nightly_ddf = ddf_visits.groupby(['target', 'day_obs_iso8601', 'filter'])['teff'].sum().reset_index()
nightly_ddf = nightly_ddf.pivot(index=['target', 'day_obs_iso8601'], columns='filter', values='teff').fillna(0.0).reset_index().set_index('target')

In [ ]:
cadence_plots = []
bands = band_cmap.transform.factors
ddf_field_names = [fn for fn in ddf_field_names if fn in nightly_ddf.index]
x_range = bokeh.models.FactorRange(factors=ddf_plot_iso8601)

for field_name in ddf_field_names:
    last_plot = len(cadence_plots) == len(ddf_field_names)-1

    df = nightly_ddf.loc[field_name, :]
    figure_kwargs = {
        'x_range' : x_range,
        'title': field_name,
        'frame_height': 150,
        'frame_width': 1024,
        'title_location': 'left',
    }

    if not last_plot:
        figure_kwargs['x_axis_location'] = None

    p = bokeh.plotting.figure(**figure_kwargs)

    p.xaxis.major_label_orientation = 'vertical'

    vbar_stack_kwargs = {
        'stackers': bands,
        'x': 'day_obs_iso8601',
        'width': 0.9,
        'source': df,
        'color': band_cmap.transform.palette,
        'fill_alpha': 0.3,
    }
    if last_plot:
        vbar_stack_kwargs['legend_label'] = bands

    p.vbar_stack(**vbar_stack_kwargs)
    if last_plot:
        legend = p.legend[0]
        legend.orientation = 'horizontal'
        p.add_layout(legend, 'below')
    
    cadence_plots.append(p)

cadence_plot_layout = bokeh.layouts.column(cadence_plots)

bokeh.io.show(cadence_plot_layout)

## Table of exposures

In [ ]:
displayed_columns = ['start_time', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime', 'numExposures', 'tau', 'skyBrightness', 'seeingFwhmEff', 'cloud', 'note']
displayed_visits_df = visits.loc[:, displayed_columns]
with pd.option_context('display.max_rows', 2000):
    display(displayed_visits_df)

In [ ]:
data_source